In [19]:
import pandas as pd

In [20]:
import numpy as np

In [21]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [22]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_17708\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [23]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_17708\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [24]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [25]:
from sklearn.preprocessing import LabelEncoder

In [26]:
le = LabelEncoder()

In [27]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [28]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [29]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [30]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [31]:
X = df_train[["Temparature","Humidity","Moisture","Crop Type","Soil Type"]].values

In [32]:
y = df_train[["Fertilizer Name"]].values

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
RFC=RandomForestClassifier()

In [35]:
fit=RFC.fit(X,y)

C:\Users\pkitt\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [36]:
RFC.fit(X,y)

C:\Users\pkitt\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [37]:
fit.score(X,y)

0.6873893333333333

In [16]:
import torch

x_tensor = torch.tensor(X, dtype=torch.float)  # X is your feature DataFrame
y_tensor = torch.tensor(y, dtype=torch.long).squeeze()  # y is your label DataFrame

In [17]:
x_fused = torch.tensor(X, dtype=torch.float)

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from torch_geometric.nn import GCNConv

# ---- Step 1: Normalize Features ----
X_np = x_fused[:275000].cpu().numpy()
scaler = StandardScaler()
X_np_scaled = scaler.fit_transform(X_np)

# ---- Step 2: Build edge_index using k-NN (n_neighbors = 10) ----
num_nodes = X_np_scaled.shape[0]
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X_np_scaled)
_, indices = nbrs.kneighbors(X_np_scaled)
rows = np.repeat(np.arange(num_nodes), indices.shape[1])
cols = indices.flatten()
edge_index_np = np.stack([rows, cols], axis=0)
edge_index = torch.from_numpy(edge_index_np).long()
edge_index = torch.cat([edge_index, edge_index[[1, 0]]], dim=1)  # Make undirected

# ---- Step 3: Prepare Tensors ----
x = torch.tensor(X_np_scaled, dtype=torch.float)
y = y_tensor[:275000]

# ---- Step 4: Device setup ----
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = x.to(device)
y = y.to(device)
edge_index = edge_index.to(device)

# ---- Step 5: Compute Class Weights ----
y_np = y.cpu().numpy()
unique_classes = np.unique(y_np)
class_weights = compute_class_weight(class_weight='balanced', classes=unique_classes, y=y_np)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# ---- Step 6: Define a Deeper GCN Model ----
class SimpleGCN(nn.Module):
    def __init__(self, in_channels, hidden_dim, num_classes):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return self.conv3(x, edge_index)

# ---- Step 7: Initialize model ----
model = SimpleGCN(in_channels=5, hidden_dim=256, num_classes=len(unique_classes)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ---- Step 8: Training Loop ----
for epoch in range(1, 4):
    model.train()
    optimizer.zero_grad()
    out = model(x, edge_index)
    loss = F.cross_entropy(out, y, weight=class_weights)
    acc = (out.argmax(dim=1) == y).float().mean().item()
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch}: Loss = {loss.item():.4f}, Accuracy = {acc:.4f}")

Epoch 1: Loss = 1.9547, Accuracy = 0.1241
Epoch 2: Loss = 1.9486, Accuracy = 0.1400
Epoch 3: Loss = 1.9475, Accuracy = 0.1478
